# Data Definition Phase

### Build MemSQL Connection

In [1]:
import pymysql
import pandas 

HOST = "127.0.0.1"
PORT = 3306
USER = "root"
PASSWORD = ""
DATABASE = "nba"

conn = pymysql.connect(
    host=HOST, 
    port=PORT, 
    user=USER, 
    password=PASSWORD, 
    database=DATABASE, 
    charset='utf8mb4'
)

### Define Function for Retrieving Individual Seasons

In [2]:
def get_games_for_season(season_start_yr, season_phase):
    """ Create a database and table for this benchmark to use. """
    season_start_yr_fmt = "%d-01-01" % season_start_yr
    cte_alias = ""

    if season_phase == "regular_season":
        cte_alias = "agirs"
        cte_name = "all_games_in_regular_season"
        cte = """
            all_games_in_regular_season AS (
                SELECT
                    *
                FROM
                    game_header gh,
                    season_dates sd
                WHERE
                    gh.game_date BETWEEN 
                        sd.regular_season_start AND 
                        sd.regular_season_end
            )
        """
    elif season_phase == "playoffs":
        cte_alias = "agip"
        cte_name = "all_games_in_playoffs"
        cte = """
            all_games_in_playoffs AS (
                SELECT
                    *
                FROM
                    game_header gh,
                    season_dates sd
                WHERE
                    gh.game_date BETWEEN 
                        sd.playoffs_start AND 
                        sd.playoffs_end
            )
        """
    elif season_phase == "finals":
        cte_alias = "agif"
        cte_name = "all_games_in_finals"
        cte = """
            all_games_in_finals AS (
                SELECT
                    *
                FROM
                    game_header gh,
                    season_dates sd
                WHERE
                    gh.game_date BETWEEN sd.finals_start AND sd.finals_end
            )
        """
    
    return pandas.read_sql_query(""" 
        WITH season_dates AS (
            SELECT
                *
            FROM season
            WHERE regular_season_start > "%(season_start_yr_fmt)s"
            ORDER BY regular_season_start ASC
            LIMIT 1
        ), %(cte)s
        SELECT
            %(cte_alias)s.game_date,
            %(cte_alias)s.natl_tv_broadcaster,
            ht.name home_team_name,
            at.name away_team_name,
            hls.pts home_team_pts,
            als.pts away_team_pts
        FROM
            %(cte_name)s %(cte_alias)s

        -- Get the home team information
        JOIN team ht ON
            %(cte_alias)s.home_team_id = ht.id
        JOIN line_score hls ON
            hls.game_id = %(cte_alias)s.game_id AND
            hls.team_id = ht.id

        -- Get the visiting team information
        JOIN team at ON
            %(cte_alias)s.away_team_id = at.id
        JOIN line_score als ON
            als.game_id = %(cte_alias)s.game_id AND
            als.team_id = at.id
        ORDER BY %(cte_alias)s.game_date DESC
    """ % {
        "season_start_yr_fmt": season_start_yr_fmt, 
        "cte": cte, 
        "cte_name": cte_name, 
        "cte_alias": cte_alias
    }, conn)

### Select a Season To Retrieve Game Data For

In [3]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# TODO: Get the years from the database
NBA_MODERN_ERA_START_YEAR=1979
NBA_SEASON_PHASES = ['regular_season', 'playoffs', 'finals']
NOW_YEAR = 2018

selected_year = NBA_MODERN_ERA_START_YEAR
selected_phase = NBA_SEASON_PHASES[0]
games_df = None

def get_games_for_year(year):
    global selected_year
    global games_df
    selected_year = year
    games_df = get_games_for_season(selected_year, selected_phase)
    print("Year:", selected_year)
    print("Phase:", selected_phase)
    print("Games in Phase:", games_df)

def set_selected_phase(phase):
    global selected_phase
    global games_df
    selected_phase = phase
    games_df = get_games_for_season(selected_year, selected_phase)
    print("Year:", selected_year)
    print("Phase:", selected_phase)
    print("Games in Phase:", games_df)
    
interact(get_games_for_year, year=widgets.IntSlider(
    min=NBA_MODERN_ERA_START_YEAR,
    max=2018,
    step=1,
    description="NBA Season",
    value=NBA_MODERN_ERA_START_YEAR
));

interact(set_selected_phase, phase=NBA_SEASON_PHASES);

interactive(children=(IntSlider(value=1979, description='NBA Season', max=2018, min=1979), Output()), _dom_cla…

interactive(children=(Dropdown(description='phase', options=('regular_season', 'playoffs', 'finals'), value='r…

## Visualize

In [5]:
import plotly.plotly as ply
from plotly.graph_objs import *

heatmap_matrix = []
teams = {}

for i, gdf in games_df.iterrows():
    home_name = gdf['home_team_name']
    away_name = gdf['away_team_name']
    home_pts = gdf['home_team_pts']
    away_pts = gdf['away_team_pts']
    
    if home_name not in teams:
        teams[home_name] = {}
    
    if away_name not in teams:
        teams[away_name] = {}
        
    if away_name not in teams[home_name]:
        teams[home_name][away_name] = 0
        
    if home_name not in teams[away_name]:

        teams[away_name][home_name] = 0

    teams[away_name][home_name] += home_pts
    teams[home_name][away_name] += away_pts

sorted_team_names = sorted(teams.keys())

layout = Layout(
     xaxis=XAxis(title='Home Team Name'),
     yaxis=YAxis(title='Away Team Name')
)

point_differentials = []
for x_team in sorted_team_names:
    z = []
    for y_team in sorted_team_names:
        if x_team == y_team:
            z.append(0)
        else:
            if y_team not in teams[x_team] or x_team not in teams[y_team]:
                delta = 0
            else:
                delta = teams[x_team][y_team] - teams[y_team][x_team]
            z.append(delta)
    point_differentials.append(z)

trace = Heatmap(
    x=sorted_team_names,
    y=sorted_team_names,
    z=point_differentials,
    colorscale='Blackbody'
)
data=[trace]
ply.iplot(data, filename='basic-heatmap')

/usr/local/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:531: DeprecationWarning:

plotly.graph_objs.XAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.XAxis
  - plotly.graph_objs.layout.scene.XAxis


/usr/local/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:558: DeprecationWarning:

plotly.graph_objs.YAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.YAxis
  - plotly.graph_objs.layout.scene.YAxis




Aw, snap! We didn't get a username with your request.

Don't have an account? https://plot.ly/api_signup

Questions? accounts@plot.ly


PlotlyError: Because you didn't supply a 'file_id' in the call, we're assuming you're trying to snag a figure from a url. You supplied the url, '', we expected it to start with 'https://plot.ly'.
Run help on this function for more information.

In [ ]:
# TODO: Get player box scores